In [1]:
%load_ext autoreload
%autoreload 2
import pinot
import torch
import numpy as np
from itertools import chain


Using backend: pytorch


In [2]:
ds = pinot.data.moonshot_meta()

In [3]:
ds_tr, ds_te = pinot.data.utils.split(ds, [4, 1])

In [18]:
def get_separate_dataset(ds):
    
    # n_tasks = ds[0][1].shape[-1]
    n_tasks = 6
    datasets = [[] for _ in range(n_tasks)]
    
    for g, y in ds:
        for idx in range(n_tasks):
            if np.isnan(y[idx].numpy()) == False:
                datasets[idx].append((g, y[idx][None]))
    
    return datasets


In [19]:
datasets = get_separate_dataset(ds_tr)
datasets = [pinot.data.utils.batch(data, len(data)) for data in datasets]

datasets_te = get_separate_dataset(ds_te)
datasets_te = [pinot.data.utils.batch(data, len(data)) for data in datasets_te]

In [21]:
nets = []

net = pinot.representation.Sequential(
    pinot.representation.dgl_legacy.gn(),
    [32, 'tanh', 32, 'tanh', 32, 'tanh'])
    
for idx in range(len(datasets)):

    
    model = pinot.inference.gp.gpr.exact_gpr.ExactGPR(
        kernel=pinot.inference.gp.kernels.deep_kernel.DeepKernel(
            base_kernel=pinot.inference.gp.kernels.rbf.RBF(scale=torch.ones(32)),
            representation=net))

    
    nets.append(model)

In [22]:
params = []

for net in nets:
    params += list(net.parameters())

opt = torch.optim.Adam(params, 1e-3)
for _ in range(500):
    opt.zero_grad()
    loss = 0.0
    for idx, net in enumerate(nets):
        g, y = datasets[idx][0]
        loss += net.loss(g, y).mean()
    print(loss)
    loss.backward()
    opt.step()
        

tensor(2644.6445, grad_fn=<AddBackward0>)
tensor(2515.0083, grad_fn=<AddBackward0>)
tensor(2431.4875, grad_fn=<AddBackward0>)
tensor(2412.8943, grad_fn=<AddBackward0>)
tensor(2382.2812, grad_fn=<AddBackward0>)
tensor(2352.2808, grad_fn=<AddBackward0>)
tensor(2331.9102, grad_fn=<AddBackward0>)
tensor(2311.9492, grad_fn=<AddBackward0>)
tensor(2291.9111, grad_fn=<AddBackward0>)
tensor(2274.6965, grad_fn=<AddBackward0>)
tensor(2246.8140, grad_fn=<AddBackward0>)
tensor(2220.9390, grad_fn=<AddBackward0>)
tensor(2220.0591, grad_fn=<AddBackward0>)
tensor(2239.3123, grad_fn=<AddBackward0>)
tensor(2229.7478, grad_fn=<AddBackward0>)
tensor(2216.8831, grad_fn=<AddBackward0>)
tensor(2201.7183, grad_fn=<AddBackward0>)
tensor(2226.2590, grad_fn=<AddBackward0>)
tensor(2193.4094, grad_fn=<AddBackward0>)
tensor(2221.0403, grad_fn=<AddBackward0>)
tensor(2188.9729, grad_fn=<AddBackward0>)
tensor(2240.0020, grad_fn=<AddBackward0>)
tensor(2214.9937, grad_fn=<AddBackward0>)
tensor(2192.6975, grad_fn=<AddBack

tensor(2105.7905, grad_fn=<AddBackward0>)
tensor(2105.9546, grad_fn=<AddBackward0>)
tensor(2099.4182, grad_fn=<AddBackward0>)
tensor(2104.9114, grad_fn=<AddBackward0>)
tensor(2106.3701, grad_fn=<AddBackward0>)
tensor(2104.1274, grad_fn=<AddBackward0>)
tensor(2103.4670, grad_fn=<AddBackward0>)
tensor(2100.1575, grad_fn=<AddBackward0>)
tensor(2098.9641, grad_fn=<AddBackward0>)
tensor(2096.6741, grad_fn=<AddBackward0>)
tensor(2096.6912, grad_fn=<AddBackward0>)
tensor(2096.8560, grad_fn=<AddBackward0>)
tensor(2099.3669, grad_fn=<AddBackward0>)
tensor(2096.3364, grad_fn=<AddBackward0>)
tensor(2099.6445, grad_fn=<AddBackward0>)
tensor(2109.3772, grad_fn=<AddBackward0>)
tensor(2099.5752, grad_fn=<AddBackward0>)
tensor(2097.1606, grad_fn=<AddBackward0>)
tensor(2104.6821, grad_fn=<AddBackward0>)
tensor(2104.8657, grad_fn=<AddBackward0>)
tensor(2100.9570, grad_fn=<AddBackward0>)
tensor(2099.5249, grad_fn=<AddBackward0>)
tensor(2107.6152, grad_fn=<AddBackward0>)
tensor(2098.3347, grad_fn=<AddBack

tensor(2110.7505, grad_fn=<AddBackward0>)
tensor(2089.3682, grad_fn=<AddBackward0>)
tensor(2051.2439, grad_fn=<AddBackward0>)
tensor(2088.2385, grad_fn=<AddBackward0>)
tensor(2102.2317, grad_fn=<AddBackward0>)
tensor(2081.1912, grad_fn=<AddBackward0>)
tensor(2059.3860, grad_fn=<AddBackward0>)
tensor(2102.1023, grad_fn=<AddBackward0>)
tensor(2126.9541, grad_fn=<AddBackward0>)
tensor(2110.5249, grad_fn=<AddBackward0>)
tensor(2064.9731, grad_fn=<AddBackward0>)
tensor(2069.7708, grad_fn=<AddBackward0>)
tensor(2079.0044, grad_fn=<AddBackward0>)
tensor(2061.0444, grad_fn=<AddBackward0>)
tensor(2058.2766, grad_fn=<AddBackward0>)
tensor(2068.1206, grad_fn=<AddBackward0>)
tensor(2059.7554, grad_fn=<AddBackward0>)
tensor(2071.0957, grad_fn=<AddBackward0>)
tensor(2067.4575, grad_fn=<AddBackward0>)
tensor(2065.3604, grad_fn=<AddBackward0>)
tensor(2063.9507, grad_fn=<AddBackward0>)
tensor(2068.1721, grad_fn=<AddBackward0>)
tensor(2070.8179, grad_fn=<AddBackward0>)
tensor(2054.5479, grad_fn=<AddBack

In [23]:
res = []


metrics = ['avg_nll', 'r2']
for idx, net in enumerate(nets):
    g, y = datasets_te[idx][0]
    
    print('-----')
    
    print(idx)
    for metric in metrics:
        _metric = getattr(pinot.metrics, metric)(net, g, y).detach().numpy()
        res.append('%.2f' % _metric)
        


-----
0
-----
1
-----
2
-----
3
-----
4
-----
5


In [24]:
print('&'.join(res))

-0.25&0.65&0.25&0.25&0.18&-0.34&0.00&0.44&0.15&0.42&0.05&0.19


In [25]:
res = []

nets = []

for idx in range(len(datasets)):

    net = pinot.representation.Sequential(
        pinot.representation.dgl_legacy.gn(),
        [32, 'tanh', 32, 'tanh', 32, 'tanh'])


    model = pinot.Net(
        net)

    nets.append(model)

    
for idx, net in enumerate(nets):
    opt = torch.optim.Adam(gp.parameters(), 1e-5)
    for _ in range(500):
        opt.zero_grad()

        g, y = datasets[idx][0]
        loss = gp.loss(g, y).mean()

        loss.backward()
        opt.step()
        
    print('-----')

    print(idx)
    for metric in metrics:
        g, y = datasets_te[idx][0]
        _metric = getattr(pinot.metrics, metric)(net, g, y).detach().numpy()
        res.append('%.2f' % _metric)
        

NameError: name 'gp' is not defined

In [58]:
print('&'.join(res))

3.48&-901.72&24999.34&-5.64&2.08&-30.52&2261755392.00&-23.28&3.80&-80.87&427.18&-467.90
